<html>
    <summary></summary>
         <div> <p></p> </div>
         <div style="font-size: 20px; width: 800px;"> 
              <h1>
               <left>Intro to Bioinformatics in Python: Downloading and Annotating Genomic Sequences.</left>
              </h1>
              <p><left>============================================================================</left> </p>
<pre>Course: BIOM/CBE 480A5, Spring 2025
Instructor: Brian Munsky
Contact Info: munsky@colostate.edu
</pre>
         </div>
    </p>

</html>

<details>
  <summary>Copyright info</summary>

```
Copyright 2024 Brian Munsky

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```
<details>



# Learning Objectives

Upon completing this tutorial, you should be able to:
- Describe the Needlemann-Wunsch algorithm for aligning genomic sequences.
- Describe the contents of a FASTA file and be able to create these in Python
- Compare and align genomic sequences using the Needleman-Wunsch algorithm.
- Translate DNA sequences into protein sequences.
- Align multiple protein sequences and calculate similarities between them.

In [ ]:
# %pip install biopython
# %pip install Bio
# %pip install pymsaviz
import Bio
from Bio import Entrez, SeqIO, pairwise2, AlignIO, Phylo
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils import ProtParam
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import webbrowser
#
import os
import threading

from Bio import Align
from Bio.Align import Alignment
from pymsaviz import MsaViz

In [4]:
#(PRELIMINARIES) Let's download the COVID19 virus sequence from NCBI
# Provide your email for accessing NCBI
Entrez.email = "a.popinga@colostate.edu"  # Insert your email here

def get_genbank(accession_number):
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

corona_virus = get_genbank("MN908947")

# 2) Comparing and Aligning Two Sequences

In [5]:
# (ALIGNMENT) Lets try to align the virus sequence with the BioNTech/Phizer and Moderna vaccines
#      First, lets create the sequences for the two vacines from the following paper: 
#      https://github.com/NAalytics/Assemblies-of-putative-SARS-CoV2-spike-encoding-mRNA-sequences-for-
#          vaccines-BNT-162b2-and-mRNA-1273/blob/main/Assemblies%20of%20putative%20SARS-CoV2-spike-encoding
#          %20mRNA%20sequences%20for%20vaccines%20BNT-162b2%20and%20mRNA-1273.docx.pdf
BNT_Phizer = '''GAGAATAAACTAGTATTCTTCTGGTCCCCACAGACTCAGAGAGAACCCGCCACCATGTTCGTGTTCCTGGTGCTGCTGCCTCTGGTGTCCA
GCCAGTGTGTGAACCTGACCACCAGAACACAGCTGCCTCCAGCCTACACCAACAGCTTTACCAGAGGCGTGTACTACCCCGACAAGGTGTT
CAGATCCAGCGTGCTGCACTCTACCCAGGACCTGTTCCTGCCTTTCTTCAGCAACGTGACCTGGTTCCACGCCATCCACGTGTCCGGCACC
AATGGCACCAAGAGATTCGACAACCCCGTGCTGCCCTTCAACGACGGGGTGTACTTTGCCAGCACCGAGAAGTCCAACATCATCAGAGGCT
GGATCTTCGGCACCACACTGGACAGCAAGACCCAGAGCCTGCTGATCGTGAACAACGCCACCAACGTGGTCATCAAAGTGTGCGAGTTCCA
GTTCTGCAACGACCCCTTCCTGGGCGTCTACTACCACAAGAACAACAAGAGCTGGATGGAAAGCGAGTTCCGGGTGTACAGCAGCGCCAAC
AACTGCACCTTCGAGTACGTGTCCCAGCCTTTCCTGATGGACCTGGAAGGCAAGCAGGGCAACTTCAAGAACCTGCGCGAGTTCGTGTTTA
AGAACATCGACGGCTACTTCAAGATCTACAGCAAGCACACCCCTATCAACCTCGTGCGGGATCTGCCTCAGGGCTTCTCTGCTCTGGAACC
CCTGGTGGATCTGCCCATCGGCATCAACATCACCCGGTTTCAGACACTGCTGGCCCTGCACAGAAGCTACCTGACACCTGGCGATAGCAGC
AGCGGATGGACAGCTGGTGCCGCCGCTTACTATGTGGGCTACCTGCAGCCTAGAACCTTCCTGCTGAAGTACAACGAGAACGGCACCATCA
CCGACGCCGTGGATTGTGCTCTGGATCCTCTGAGCGAGACAAAGTGCACCCTGAAGTCCTTCACCGTGGAAAAGGGCATCTACCAGACCAG
CAACTTCCGGGTGCAGCCCACCGAATCCATCGTGCGGTTCCCCAATATCACCAATCTGTGCCCCTTCGGCGAGGTGTTCAATGCCACCAGA
TTCGCCTCTGTGTACGCCTGGAACCGGAAGCGGATCAGCAATTGCGTGGCCGACTACTCCGTGCTGTACAACTCCGCCAGCTTCAGCACCT
TCAAGTGCTACGGCGTGTCCCCTACCAAGCTGAACGACCTGTGCTTCACAAACGTGTACGCCGACAGCTTCGTGATCCGGGGAGATGAAGT
GCGGCAGATTGCCCCTGGACAGACAGGCAAGATCGCCGACTACAACTACAAGCTGCCCGACGACTTCACCGGCTGTGTGATTGCCTGGAAC
AGCAACAACCTGGACTCCAAAGTCGGCGGCAACTACAATTACCTGTACCGGCTGTTCCGGAAGTCCAATCTGAAGCCCTTCGAGCGGGACA
TCTCCACCGAGATCTATCAGGCCGGCAGCACCCCTTGTAACGGCGTGGAAGGCTTCAACTGCTACTTCCCACTGCAGTCCTACGGCTTTCA
GCCCACAAATGGCGTGGGCTATCAGCCCTACAGAGTGGTGGTGCTGAGCTTCGAACTGCTGCATGCCCCTGCCACAGTGTGCGGCCCTAAG
AAAAGCACCAATCTCGTGAAGAACAAATGCGTGAACTTCAACTTCAACGGCCTGACCGGCACCGGCGTGCTGACAGAGAGCAACAAGAAGT
TCCTGCCATTCCAGCAGTTTGGCCGGGATATCGCCGATACCACAGACGCCGTTAGAGATCCCCAGACACTGGAAATCCTGGACATCACCCC
TTGCAGCTTCGGCGGAGTGTCTGTGATCACCCCTGGCACCAACACCAGCAATCAGGTGGCAGTGCTGTACCAGGACGTGAACTGTACCGAA
GTGCCCGTGGCCATTCACGCCGATCAGCTGACACCTACATGGCGGGTGTACTCCACCGGCAGCAATGTGTTTCAGACCAGAGCCGGCTGTC
TGATCGGAGCCGAGCACGTGAACAATAGCTACGAGTGCGACATCCCCATCGGCGCTGGAATCTGCGCCAGCTACCAGACACAGACAAACAG
CCCTCGGAGAGCCAGAAGCGTGGCCAGCCAGAGCATCATTGCCTACACAATGTCTCTGGGCGCCGAGAACAGCGTGGCCTACTCCAACAAC
TCTATCGCTATCCCCACCAACTTCACCATCAGCGTGACCACAGAGATCCTGCCTGTGTCCATGACCAAGACCAGCGTGGACTGCACCATGT
ACATCTGCGGCGATTCCACCGAGTGCTCCAACCTGCTGCTGCAGTACGGCAGCTTCTGCACCCAGCTGAATAGAGCCCTGACAGGGATCGC
CGTGGAACAGGACAAGAACACCCAAGAGGTGTTCGCCCAAGTGAAGCAGATCTACAAGACCCCTCCTATCAAGGACTTCGGCGGCTTCAAT
TTCAGCCAGATTCTGCCCGATCCTAGCAAGCCCAGCAAGCGGAGCTTCATCGAGGACCTGCTGTTCAACAAAGTGACACTGGCCGACGCCG
GCTTCATCAAGCAGTATGGCGATTGTCTGGGCGACATTGCCGCCAGGGATCTGATTTGCGCCCAGAAGTTTAACGGACTGACAGTGCTGCC
TCCTCTGCTGACCGATGAGATGATCGCCCAGTACACATCTGCCCTGCTGGCCGGCACAATCACAAGCGGCTGGACATTTGGAGCAGGCGCC
GCTCTGCAGATCCCCTTTGCTATGCAGATGGCCTACCGGTTCAACGGCATCGGAGTGACCCAGAATGTGCTGTACGAGAACCAGAAGCTGA
TCGCCAACCAGTTCAACAGCGCCATCGGCAAGATCCAGGACAGCCTGAGCAGCACAGCAAGCGCCCTGGGAAAGCTGCAGGACGTGGTCAA
CCAGAATGCCCAGGCACTGAACACCCTGGTCAAGCAGCTGTCCTCCAACTTCGGCGCCATCAGCTCTGTGCTGAACGATATCCTGAGCAGA
CTGGACCCTCCTGAGGCCGAGGTGCAGATCGACAGACTGATCACAGGCAGACTGCAGAGCCTCCAGACATACGTGACCCAGCAGCTGATCA
GAGCCGCCGAGATTAGAGCCTCTGCCAATCTGGCCGCCACCAAGATGTCTGAGTGTGTGCTGGGCCAGAGCAAGAGAGTGGACTTTTGCGG
CAAGGGCTACCACCTGATGAGCTTCCCTCAGTCTGCCCCTCACGGCGTGGTGTTTCTGCACGTGACATATGTGCCCGCTCAAGAGAAGAAT
TTCACCACCGCTCCAGCCATCTGCCACGACGGCAAAGCCCACTTTCCTAGAGAAGGCGTGTTCGTGTCCAACGGCACCCATTGGTTCGTGA
CACAGCGGAACTTCTACGAGCCCCAGATCATCACCACCGACAACACCTTCGTGTCTGGCAACTGCGACGTCGTGATCGGCATTGTGAACAA
TACCGTGTACGACCCTCTGCAGCCCGAGCTGGACAGCTTCAAAGAGGAACTGGACAAGTACTTTAAGAACCACACAAGCCCCGACGTGGAC
CTGGGCGATATCAGCGGAATCAATGCCAGCGTCGTGAACATCCAGAAAGAGATCGACCGGCTGAACGAGGTGGCCAAGAATCTGAACGAGA
GCCTGATCGACCTGCAAGAACTGGGGAAGTACGAGCAGTACATCAAGTGGCCCTGGTACATCTGGCTGGGCTTTATCGCCGGACTGATTGC
CATCGTGATGGTCACAATCATGCTGTGTTGCATGACCAGCTGCTGTAGCTGCCTGAAGGGCTGTTGTAGCTGTGGCAGCTGCTGCAAGTTC
GACGAGGACGATTCTGAGCCCGTGCTGAAGGGCGTGAAACTGCACTACACATGATGACTCGAGCTGGTACTGCATGCACGCAATGCTAGCT
GCCCCTTTCCCGTCCTGGGTACCCCGAGTCTCCCCCGACCTCGGGTCCCAGGTATGCTCCCACCTCCACCTGCCCCACTCACCACCTCTGC
TAGTTCCAGACACCTCCCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGATTAACCTTT
AGCAATAAACGAAAGTTTAACTAAGCTATACTAACCCCAGGGTTGGTCAATTTCGTGCCAGCCACACCCTGGAGCTAGCA'''
BNT_Phizer = BNT_Phizer.replace("\n", "")

Moderna = '''GGGAAATAAGAGAGAAAAGAAGAGTAAGAAGAAATATAAGACCCCGGCGCCGCCACCATGTTCGTGTTCCTGGTGCTGCTGCCCCTGGTGA
GCAGCCAGTGCGTGAACCTGACCACCCGGACCCAGCTGCCACCAGCCTACACCAACAGCTTCACCCGGGGCGTCTACTACCCCGACAAGGT
GTTCCGGAGCAGCGTCCTGCACAGCACCCAGGACCTGTTCCTGCCCTTCTTCAGCAACGTGACCTGGTTCCACGCCATCCACGTGAGCGGC
ACCAACGGCACCAAGCGGTTCGACAACCCCGTGCTGCCCTTCAACGACGGCGTGTACTTCGCCAGCACCGAGAAGAGCAACATCATCCGGG
GCTGGATCTTCGGCACCACCCTGGACAGCAAGACCCAGAGCCTGCTGATCGTGAATAACGCCACCAACGTGGTGATCAAGGTGTGCGAGTT
CCAGTTCTGCAACGACCCCTTCCTGGGCGTGTACTACCACAAGAACAACAAGAGCTGGATGGAGAGCGAGTTCCGGGTGTACAGCAGCGCC
AACAACTGCACCTTCGAGTACGTGAGCCAGCCCTTCCTGATGGACCTGGAGGGCAAGCAGGGCAACTTCAAGAACCTGCGGGAGTTCGTGT
TCAAGAACATCGACGGCTACTTCAAGATCTACAGCAAGCACACCCCAATCAACCTGGTGCGGGATCTGCCCCAGGGCTTCTCAGCCCTGGA
GCCCCTGGTGGACCTGCCCATCGGCATCAACATCACCCGGTTCCAGACCCTGCTGGCCCTGCACCGGAGCTACCTGACCCCAGGCGACAGC
AGCAGCGGGTGGACAGCAGGCGCGGCTGCTTACTACGTGGGCTACCTGCAGCCCCGGACCTTCCTGCTGAAGTACAACGAGAACGGCACCA
TCACCGACGCCGTGGACTGCGCCCTGGACCCTCTGAGCGAGACCAAGTGCACCCTGAAGAGCTTCACCGTGGAGAAGGGCATCTACCAGAC
CAGCAACTTCCGGGTGCAGCCCACCGAGAGCATCGTGCGGTTCCCCAACATCACCAACCTGTGCCCCTTCGGCGAGGTGTTCAACGCCACC
CGGTTCGCCAGCGTGTACGCCTGGAACCGGAAGCGGATCAGCAACTGCGTGGCCGACTACAGCGTGCTGTACAACAGCGCCAGCTTCAGCA
CCTTCAAGTGCTACGGCGTGAGCCCCACCAAGCTGAACGACCTGTGCTTCACCAACGTGTACGCCGACAGCTTCGTGATCCGTGGCGACGA
GGTGCGGCAGATCGCACCCGGCCAGACAGGCAAGATCGCCGACTACAACTACAAGCTGCCCGACGACTTCACCGGCTGCGTGATCGCCTGG
AACAGCAACAACCTCGACAGCAAGGTGGGCGGCAACTACAACTACCTGTACCGGCTGTTCCGGAAGAGCAACCTGAAGCCCTTCGAGCGGG
ACATCAGCACCGAGATCTACCAAGCCGGCTCCACCCCTTGCAACGGCGTGGAGGGCTTCAACTGCTACTTCCCTCTGCAGAGCTACGGCTT
CCAGCCCACCAACGGCGTGGGCTACCAGCCCTACCGGGTGGTGGTGCTGAGCTTCGAGCTGCTGCACGCCCCAGCCACCGTGTGTGGCCCC
AAGAAGAGCACCAACCTGGTGAAGAACAAGTGCGTGAACTTCAACTTCAACGGCCTTACCGGCACCGGCGTGCTGACCGAGAGCAACAAGA
AATTCCTGCCCTTTCAGCAGTTCGGCCGGGACATCGCCGACACCACCGACGCTGTGCGGGATCCCCAGACCCTGGAGATCCTGGACATCAC
CCCTTGCAGCTTCGGCGGCGTGAGCGTGATCACCCCAGGCACCAACACCAGCAACCAGGTGGCCGTGCTGTACCAGGACGTGAACTGCACC
GAGGTGCCCGTGGCCATCCACGCCGACCAGCTGACACCCACCTGGCGGGTCTACAGCACCGGCAGCAACGTGTTCCAGACCCGGGCCGGTT
GCCTGATCGGCGCCGAGCACGTGAACAACAGCTACGAGTGCGACATCCCCATCGGCGCCGGCATCTGTGCCAGCTACCAGACCCAGACCAA
TTCACCCCGGAGGGCAAGGAGCGTGGCCAGCCAGAGCATCATCGCCTACACCATGAGCCTGGGCGCCGAGAACAGCGTGGCCTACAGCAAC
AACAGCATCGCCATCCCCACCAACTTCACCATCAGCGTGACCACCGAGATTCTGCCCGTGAGCATGACCAAGACCAGCGTGGACTGCACCA
TGTACATCTGCGGCGACAGCACCGAGTGCAGCAACCTGCTGCTGCAGTACGGCAGCTTCTGCACCCAGCTGAACCGGGCCCTGACCGGCAT
CGCCGTGGAGCAGGACAAGAACACCCAGGAGGTGTTCGCCCAGGTGAAGCAGATCTACAAGACCCCTCCCATCAAGGACTTCGGCGGCTTC
AACTTCAGCCAGATCCTGCCCGACCCCAGCAAGCCCAGCAAGCGGAGCTTCATCGAGGACCTGCTGTTCAACAAGGTGACCCTAGCCGACG
CCGGCTTCATCAAGCAGTACGGCGACTGCCTCGGCGACATAGCCGCCCGGGACCTGATCTGCGCCCAGAAGTTCAACGGCCTGACCGTGCT
GCCTCCCCTGCTGACCGACGAGATGATCGCCCAGTACACCAGCGCCCTGTTAGCCGGAACCATCACCAGCGGCTGGACTTTCGGCGCTGGA
GCCGCTCTGCAGATCCCCTTCGCCATGCAGATGGCCTACCGGTTCAACGGCATCGGCGTGACCCAGAACGTGCTGTACGAGAACCAGAAGC
TGATCGCCAACCAGTTCAACAGCGCCATCGGCAAGATCCAGGACAGCCTGAGCAGCACCGCTAGCGCCCTGGGCAAGCTGCAGGACGTGGT
GAACCAGAACGCCCAGGCCCTGAACACCCTGGTGAAGCAGCTGAGCAGCAACTTCGGCGCCATCAGCAGCGTGCTGAACGACATCCTGAGC
CGGCTGGACCCTCCCGAGGCCGAGGTGCAGATCGACCGGCTGATCACTGGCCGGCTGCAGAGCCTGCAGACCTACGTGACCCAGCAGCTGA
TCCGGGCCGCCGAGATTCGGGCCAGCGCCAACCTGGCCGCCACCAAGATGAGCGAGTGCGTGCTGGGCCAGAGCAAGCGGGTGGACTTCTG
CGGCAAGGGCTACCACCTGATGAGCTTTCCCCAGAGCGCACCCCACGGAGTGGTGTTCCTGCACGTGACCTACGTGCCCGCCCAGGAGAAG
AACTTCACCACCGCCCCAGCCATCTGCCACGACGGCAAGGCCCACTTTCCCCGGGAGGGCGTGTTCGTGAGCAACGGCACCCACTGGTTCG
TGACCCAGCGGAACTTCTACGAGCCCCAGATCATCACCACCGACAACACCTTCGTGAGCGGCAACTGCGACGTGGTGATCGGCATCGTGAA
CAACACCGTGTACGATCCCCTGCAGCCCGAGCTGGACAGCTTCAAGGAGGAGCTGGACAAGTACTTCAAGAATCACACCAGCCCCGACGTG
GACCTGGGCGACATCAGCGGCATCAACGCCAGCGTGGTGAACATCCAGAAGGAGATCGATCGGCTGAACGAGGTGGCCAAGAACCTGAACG
AGAGCCTGATCGACCTGCAGGAGCTGGGCAAGTACGAGCAGTACATCAAGTGGCCCTGGTACATCTGGCTGGGCTTCATCGCCGGCCTGAT
CGCCATCGTGATGGTGACCATCATGCTGTGCTGCATGACCAGCTGCTGCAGCTGCCTGAAGGGCTGTTGCAGCTGCGGCAGCTGCTGCAAG
TTCGACGAGGACGACAGCGAGCCCGTGCTGAAGGGCGTGAAGCTGCACTACACCTGATAATAGGCTGGAGCCTCGGTGGCCTAGCTTCTTG
CCCCTTGGGCCTCCCCCCAGCCCCTCCTCCCCTTCCTGCACCCGTACCCCCGTGGTCTTTGAATAAAGTCTGAGTGGGCGGCAAAAAAAAA'''

Moderna = Moderna.replace("\n", "")

## 2.A) Aligning nucleotide sequences using the Needleman-Wunsch Algorithm
Now that we have two sequences, let's align them (arrange them to maximize similarities in their sequences). To do this, we are going to use the **Needleman-Wunsch algorithm**.

The **Needleman-Wunsch algorithm** is a **dynamic programming** approach for global sequence alignment. It aligns two sequences, *S1*​ and *S2*, by assigning scores for matches, mismatches, and gaps. The goal is to find the optimal alignment that maximizes the cumulative score.

### Scoring Scheme:

Let:

* $s$ be the *match score*,
* $d$ be the *mismatch score*,
* $g$ be the *gap penalty*.

The scoring matrix, $\mathbf{M}$, is initialized as an $n\times m$ matrix, where $n$ is the length of sequence *S1** and $m$ is the length of sequence *S2*.

### Initialization:
At the start of the algorithm, we initialize the scoring matrix as:

$M_{i,0}=i\cdot g$ for $i=0,1,\ldots,n$

$M_{0,j}=j\cdot g$ for $j=0,1,\ldots,m$

### Recurrence Relation:

The entries of the scoring matrix are filled using the recurrence relation:

$M_{i,j} =\max\left[
\begin{matrix}  M_{i−1,j−1} + s &\textrm{ if } S1[i]=S2[j] &\textrm{ (match)}\\
  M_{i−1,j−1} + d &\textrm{ if } S1[i]\ne S2[j] &\textrm{ (mis-match)}\\
  M_{i−1,j} + g  &&\textrm{ (gap in S1)}\\
  M_{i,j-1} + g  &&\textrm{ (gap in S2)}
  \end{matrix}\right]$

### Traceback

After filling the scoring matrix, the optimal alignment is found by traceback through the matrix. Starting from the bottom-right corner of the matrix, the traceback follows the maximum score path until reaching the top-left corner.


Alignment=(aligned sequence *S1* , aligned sequence $S2$)

### Example Needleman-Wunsch Algorithm Code:

In [6]:
import numpy as np

def needleman_wunsch(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-1, verbose=False):
    # Initialize the scoring matrix
    rows, cols = len(seq1) + 1, len(seq2) + 1
    score_matrix = np.zeros((rows, cols), dtype=int)

    # Initialize the scoring matrix with gap penalties
    for i in range(rows):
        score_matrix[i, 0] = i * gap_penalty
    for j in range(cols):
        score_matrix[0, j] = j * gap_penalty

    if verbose:
        # Print the scoring matrix again, but with the sequences overlayed.
        # This is a bit trickier because we need to pad the sequences to make them the same length
        print("Initial Scoring Matrix:")
        print("     " + "  ".join(seq2))
        for i in range(rows):
            print(seq1[i-1] if i > 0 else " ", " ".join(str(score_matrix[i, j]) for j in range(cols)))

    # Fill the scoring matrix using the recurrence relationship
    for i in range(1, rows):
        for j in range(1, cols):
            match = score_matrix[i-1, j-1] + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score)
            delete = score_matrix[i-1, j] + gap_penalty
            insert = score_matrix[i, j-1] + gap_penalty
            score_matrix[i, j] = max(match, delete, insert)

    if verbose:
        # Print the scoring matrix again, but with the sequences overlayed.
        # This is a bit trickier because we need to pad the sequences to make them the same length
        print("\nFinal Scoring Matrix:")
        print("     " + "  ".join(seq2))
        for i in range(rows):
            print(seq1[i-1] if i > 0 else " ", " ".join(str(score_matrix[i, j]) for j in range(cols)))

    # Traceback to find the alignment of the two sequences
    align1, align2 = '', ''
    i, j = rows - 1, cols - 1

    while i > 0 or j > 0:
        current_score = score_matrix[i, j]
        
        # Calculate the score for each possible direction (i.e., diagonal, up, left)
        # If we are at the top or left edge, we need to set the score for that direction to negative infinity
        diagonal = score_matrix[i-1, j-1] if i > 0 and j > 0 else float('-inf')
        up = score_matrix[i-1, j] if i > 0 else float('-inf')
        left = score_matrix[i, j-1] if j > 0 else float('-inf')
        
        if current_score == diagonal + (match_score if seq1[i-1] == seq2[j-1] else mismatch_score):
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
        elif current_score == up + gap_penalty:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1

    return align1, align2

#### Example usage
Now let's try it out using a couple simple sequences:


In [ ]:
# seq1 = "AGTACGCA"
# seq2 = "TATGC"
seq1 = "GGATCGAG"
seq2 = "GACCG"
alignment1, alignment2 = needleman_wunsch(seq1, seq2, verbose=True)

print("\nSequence 1:", alignment1)
print("Sequence 2:", alignment2)

alignment = pairwise2.align.globalms(seq1, seq2, 1, -1, -1, -1)[0]
print(alignment)

In [ ]:
# Now let's align our two vaccine sequences using the Needleman-Wunsch algorithm we just wrote
alignment1, alignment2 = needleman_wunsch(BNT_Phizer, Moderna, verbose=False)

print("\nSequence 1:", alignment1)
print("Sequence 2:", alignment2)

# This routine could take a minute or two to run.  There are more efficient ways to do this, but this is simple.
# Later we will use Muscle5 to do this much more efficiently.

## 2.B) Aligning sequences using functions from the Bio module

The ```Bio``` module also contains a more flexible alignment function ```pairwise2```.  Let's use the python help function to get some more information about that function.

In [ ]:
pairwise2.align.globalms?

Let's try it out.

In [ ]:
# Perform sequence alignment
alignment_BNT_MOD = pairwise2.align.globalxx(BNT_Phizer, Moderna)[0]
alignment_VIR_MOD = pairwise2.align.globalxx(corona_virus.seq, Moderna)[0]
alignment_VIR_BNT = pairwise2.align.globalxx(corona_virus.seq, BNT_Phizer)[0]

# Define a simple function for visualizing sequence alignments
def visualize_alignment(alignment, headers):
    seq1, seq2, score, begin, end = alignment
    print(f"Alignment Score: {score}")

# Display alignments
print("Alignment between BNT_Phizer and Moderna:")
visualize_alignment(alignment_BNT_MOD, ['BioNTech', 'Moderna'])

print("\nAlignment between Virus and Moderna:")
visualize_alignment(alignment_VIR_MOD, ['Virus', 'Moderna'])

print("\nAlignment between Virus and BNT_Phizer:")
visualize_alignment(alignment_VIR_BNT, ['Virus', 'BioNTech'])

# This cell could also take a minute or two to run.

## 2.C) Saving alignments as FASTA files

FASTA files are text-based files used to represent nucleotide sequences or peptide sequences. Each sequence in a FASTA file is preceded by a single-line description, which starts with a greater-than (">") symbol. The description line is followed by lines of sequence data.

**Structure of a FASTA file:**
1. **Header Line**: Begins with a ">" symbol followed by a description or identifier of the sequence.
2. **Sequence Lines**: One or more lines of sequence data (nucleotide or protein sequences).

**Example:**
   $>$ sequence1
   
   AGCTGATCGATCGTACGATCG
   
   $>$ sequence2

   CGTAGCTAGCTAGCTAGCTAG

**Key Features of FASTA Files:**
- **Header Line**: Text proceeded with $>$ that provides a description or identifier for the sequence.
- **Sequence Data**: Text containing the actual sequence, which can span multiple lines.
- **File Extension**: Typically uses `.fasta` or `.fa`.

FASTA files are widely used in bioinformatics for storing and sharing sequence data, and we will use them throughout this module. 

Let's save our results to a FASTA file:

In [ ]:
# Create SeqRecord objects for the aligned sequences
seq_record1 = SeqRecord(Seq(alignment_BNT_MOD[0]), id='BioNTech')
seq_record2 = SeqRecord(Seq(alignment_BNT_MOD[1]), id='Moderna')

# Save alignment to a .fa file
alignment_records = [seq_record1, seq_record2]
output_file = 'alignment_results.fa'
SeqIO.write(alignment_records, output_file, 'fasta')


Let's see what the FASTA file looks like. 

In [ ]:
# Print the first 10 lines of the alignment file
with open('alignment_results.fa', 'r') as f:
    for i in range(10):
        print(f.readline().strip())



In the FASTA alignment file, the first line is the header for the first sequence, and the second line is the beginning of the first sequence itself (in this case the BioNTech vaccine).

Here, the alignment shows letters G, A, T, C, and - (representing gaps).

### Browsing FASTA contents using command line tools

Sometimes you may want to quickly see which seqeuences are in a FASTA file. We can use some simple command line functions to do this:

```echo "aligned_sequences.fa" | xargs grep ">"```

Here the command `echo "aligned_sequences.fa"` prints the name of the file to the terminal, and the `|` character is a pipe that sends the output of the `echo` command to the `grep` command. The `grep` command searches for lines in a file that match a given pattern, and in this case we are searching for lines that start with `>`, which is the standard way to denote a sequence header in a FASTA file.

Try it out in the terminal or by using a magic command like in the next cell.

In [ ]:
!echo "aligned_sequences.fa" | xargs grep ">"

## 2.C) Visualizing sequence allignments
Now that we have the sequences aligned to one another, it is very helpful to generate a plot to visualize the two sequence's similarities.  Let's make a plot and save it to a file.

Here, we are going to create a figure of the alignment using the ```MsaViz``` function.  This could take a couple minutes to run if we have many or long sequences.

In [14]:
# Visualize the alignment using MsaViz.
msa_file = "alignment_results.fa"
mv = MsaViz(msa_file, wrap_length=200, show_count=True)
mv.savefig("BioNTech_vs_Moderna_Alignment.png")

#### Loading and displaying an image
When MsaViz completes, it will have created a PNG image of the alignment that we can load to look at our results.

In [ ]:
# Load the image
img = mpimg.imread('BioNTech_vs_Moderna_Alignment.png')

# Display the image
fig = plt.figure(figsize=(30, 30), dpi=200)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()

# 3) Comparing and aligning amino acid sequences

So far, we have focused on aligning nucleic acid sequences, but we could have tried to align the protein sequences instead.  Aligning amino acid sequences is often preferred over nucleic acid sequences for several reasons:

1. **Higher Information Content**: Amino acids have 20 different types, compared to only 4 nucleotides. This higher complexity provides more information and can lead to more accurate alignments.

2. **Functional Relevance**: Proteins (composed of amino acids) are the functional molecules in cells. Aligning amino acid sequences can provide more direct insights into functional and structural similarities of those proteins.

3. **Evolutionary Conservation**: Amino acid sequences are more conserved than nucleotide sequences. This means that functionally important regions are more likely to be preserved, making it easier to identify homologous regions between species.

4. **Codon Degeneracy**: Multiple nucleotide sequences can code for the same amino acid due to the redundancy in the genetic code. Aligning amino acids bypasses this redundancy and focuses on the actual protein sequence. Aligning amino acids reduces this noise and highlights more significant evolutionary changes.

Overall, aligning amino acid sequences can provide more meaningful and accurate insights into the evolutionary and functional relationships between sequences.

But before we can align the amino acid sequences, we need to find those sequences!  

To do that let's first try to find all of the ```Open Reading Frames (Orfs)``` and then pick the largest one.

In [16]:
# We can use our code from our previous notebook to find the ORFs
def find_all_orfs(sequence):
    orf_list = []           # List of all the ORF we find
    for frame in range(3):  # Check each of the three reading frames
        orfs = []           # List of ORFs for the current frame   
        lastStop = frame    # Position of last stop codon.
        for orf in sequence[frame:].translate(to_stop=False).split('*'):     # Translate the curent frame sequence into AA and break up by stop codons
            if len(orf) > 30:                                                # Consider ORFs longer than 50 amino acids
                start = lastStop + orf.find('M') * 3 if 'M' in orf else None # Update start codon position to next M, 
                stop = lastStop + (len(orf))*3 if 'M' in orf else None       # Update stop codon to next '*'
                if start is not None and stop is not None:                   # In the coding region is >30AA
                    orfs.append((start, stop))                                      # Append the start and stop positions for that ORF
            lastStop += len(orf)*3 + 3           # Update the position of the last stop codon
        orf_list.append(orfs)                    # Append the list of ORFs for the current frame
    return orf_list

In [17]:
# Find all ORFs in each frame of the vaccine sequences
seqModerna = Seq(Moderna)
seqBNTPhizer = Seq(BNT_Phizer)

Moderna_orf_positions = find_all_orfs(seqModerna)
BNT_Phizer_orf_positions = find_all_orfs(seqBNTPhizer)

In [ ]:
# Next, let's find the amino acid sequence for the largest open reading frame from each vaccine
def pickLargestORF(orfs):
    maxLength = 0;
    for frame, orfs in enumerate(orfs):
        for start, stop in orfs:
            if (stop-start)>maxLength:
                largestStart = start
                largestStop = stop
                maxLength = stop-start
            
    return largestStart, largestStop

# Find the start and stop positions of the largest ORF for each vaccine
startMod, stopMod = pickLargestORF(Moderna_orf_positions)
ModernaAA = seqModerna[startMod:stopMod].translate(to_stop=False)
print(ModernaAA)

startBNT, stopBNT = pickLargestORF(BNT_Phizer_orf_positions)
BNTPhiAA = seqBNTPhizer[startBNT:stopBNT].translate(to_stop=False)
print(BNTPhiAA)

In [ ]:
#Now, lets align the vaccine amino acid sequences to one another. For this, we are just going to copy some of 
# the codes from before.

# Perform sequence alignment
alignment_BNTaa_MODaa = pairwise2.align.globalxx(BNTPhiAA, ModernaAA)[0]

# Create SeqRecord objects for the aligned sequences
seq_record1 = SeqRecord(Seq(alignment_BNTaa_MODaa[0]), id='BioNTech')
seq_record2 = SeqRecord(Seq(alignment_BNTaa_MODaa[1]), id='Moderna')

# Save alignment to a .fa file
alignment_records = [seq_record1, seq_record2]
output_file = 'alignment_results_AA.fa'
SeqIO.write(alignment_records, output_file, 'fasta')

# Make and save an alignment figure
mv = MsaViz(output_file, wrap_length=100, show_count=True)
mv.savefig("BioNTech_vs_Moderna_Alignment.png")

# Load the image
img = mpimg.imread('BioNTech_vs_Moderna_Alignment.png')

# Display the image
fig = plt.figure(figsize=(30, 30), dpi=200)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()

Now we see that the two vaccines are almost identical in their amino acid sequence.  That is way too unlikely to be a coincidence!  We will need to compare to the corona virus sequence itself to see why the designers chose that particular sequence.

For this, we now need to go beyond pairwise aligments and consider **multiple sequence aligment.**

# 4) Comparing and Aligning Multiple Protein Sequences
Now, let's see how the Moderna and BNT/Phizer vaccinnes compare to the original virus sequence.  

To do this, we are going to start by finding the 10 largest ORFs in the corona virus genome.

Like last time, we need to get the sequence from genbank and extract its ORFs.

In [20]:
# Let's download the COVID19 virus sequence from NCBI
# Provide your email for accessing NCBI
Entrez.email = "brian.munsky@colostate.edu"  # Insert your email here

def get_genbank(accession_number):
    handle = Entrez.efetch(db="nucleotide", id=accession_number, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    return record

corona_virus = get_genbank("MN908947")

In [21]:
# Find all ORFs in each frame of the sequence
orf_positions = find_all_orfs(corona_virus.seq)

In [22]:
# First, we will create a function that will allow us to pick the N largest ORFs from a list of ORFs,
def pickNLargestORF(orfs,N=5):
    Lengths = []
    Bounds = []
    for frame, orf in enumerate(orfs):
        for start, stop in orf:
            Lengths.append(stop-start)
            Bounds.append([start,stop])
    enumerated_list = list(enumerate(Lengths))
    sortedLengths = sorted(enumerated_list, key=lambda x: abs(x[1]), reverse=True)

    # # Extract indices of the N largest magnitude entries
    # largest_indices = [index for index, _ in sortedLengths[:N]]
    
    # # Extract lengths and bounds for those indices
    # longestLengths = [Lengths[index] for index, _ in sortedLengths[:N]]
    Bounds = [Bounds[index] for index, _ in sortedLengths[:N]]
    
    return Bounds


In [ ]:
# Pick the largest ORF and translate it to amino acids
longestORFbounds = pickNLargestORF(orf_positions,10)
longestORFsequences = [corona_virus.seq[longestORFbounds[index][0]:].translate(to_stop=True) for index in range(10)]
print(longestORFsequences)

## 4.A) Running MUSCLE to align sequences with a command line call

For multisequence alignments, we are going to want to use another, faster program that needs to be
called from the command line. This can be done from python using command line (BASH). 

Here, we a going to use the program ```MUSCLE5``` to align our sequences (the two vacines and the 10 largest corona virus proteins). 

But first, before we can align the sequences, we need to put them together into a FASTA file.

In [24]:
# Create a FASTA file with all of the un-aligned sequences
seqFile = "sequences.fa"

# Create a FASTA file with sequences
with open(seqFile, "w") as handle:
    handle.write(f">Moderna\n{ModernaAA}\n")
    handle.write(f">BNTPhi\n{BNTPhiAA}\n")
    for idx in enumerate(longestORFsequences):
        handle.write(f">seq" + str(idx[0]) + "\n" + str(idx[1]) + "\n"), 

In [ ]:
# Let's run our command line code again to make sure the sequences are in the file
!echo "sequences.fa" | xargs grep ">"

Now that we have our FASTA file, let's call Muscle5 to solve for its alignment.  To call muscle, we are going to use 

```muscle -align sequences.fa -output aligned_sequences.fa```

We could run that directly in the terminal, but here we show how to do it directly from the notebook using the ```os.system``` function with a ```bash``` command:

In [ ]:
# Choose the file name for the aligned sequences
alignedFile = "aligned_sequences.fa"

# Create a string that has the full bash command
bashCommand = "muscle -align " + seqFile + " -output " + alignedFile
print(f'We are about to run the command:\n{bashCommand}')

# Run the command
!{bashCommand}    # or os.system(bashCommand)

# If you get an error here, you may need to install muscle, or make sure it is in your path.
# Please see the instructions on Canvas for how to install muscle.
# To see what is in your path, you can run the following command:
# print(os.environ['PATH'])
# or
# !echo $PATH
# If you need to add a directory to your path, you can do so with the following command:
# os.environ['PATH'] += os.pathsep + '/path/to/muscle'


In [28]:
# Make and save an alignment figure
mv = MsaViz(alignedFile, wrap_length=100, show_count=True)
mv.savefig("BioNTech_vs_Moderna_Alignment.png")

# This could take a couple minutes to run.  
# If it takes too long, you can kill it by clicking the stop button in the toolbar above.

# All this function is doing is taking the aligned sequences and making a pretty figure out of them.
# You can see the alignments in the file "aligned_sequences.fa" if you want to look at them directly,
# so you can skip the MsaViz step until you are trying to make final, pretty figures.

In [ ]:
# Next, we load and print the image to the screen.

# Load the image
img = mpimg.imread('BioNTech_vs_Moderna_Alignment.png')

# Display the image
fig = plt.figure(figsize=(200, 80), dpi=200)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()

If you look at the figure that was made, you will eventually see that the two vaccines align very closely to each other and also to sequence 2.  But could we have got this information more easily without having to squint?

## 4.B) Calculating Alignment Similarities

Once we have the aligned FASTA files, it is very easy to see how closely each sequence aligns with the others.  We will just compute the pairwise differences between each sequence.

In [ ]:
# Load the aligned sequences and compute the pairwise identity between them.
# We will use the Bio.pairwise2 module to do this.
# This module is part of the Biopython package.

# Load the aligned sequences
aligned_sequences = []
seq_names = []
with open(alignedFile, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        aligned_sequences.append(record.seq)
        seq_names.append(record.id)

# Compute the pairwise identity between all pairs of sequences
pairwise_identity = np.zeros((len(aligned_sequences), len(aligned_sequences)))
for i in range(len(aligned_sequences)):
    for j in range(len(aligned_sequences)):
        pairwise_identity[i,j] = sum([1 for a, b in zip(aligned_sequences[i], aligned_sequences[j]) if a == b and a != '-']) / sum([1 for a in aligned_sequences[i] if a != '-'])

# Print the pairwise identity to three significant digits.
print(np.round(pairwise_identity,3))

In [ ]:
# Make a heatmap plot of the pairwise distances.
plt.figure(figsize=(10, 10))
plt.imshow(pairwise_identity, cmap='viridis')
plt.colorbar(label='Pairwise Identity')
plt.xlabel('Sequence Index')
plt.ylabel('Sequence Index')
plt.title('Pairwise Identity Between Sequences')
# Put the names of the sequences on the axes
plt.xticks(range(len(aligned_sequences)), [f"{seq_names[i]}" for i in range(len(aligned_sequences))], rotation=90)
plt.yticks(range(len(aligned_sequences)), [f"{seq_names[i]}" for i in range(len(aligned_sequences))])
plt.show()


Looking at the figure, we quickly see that seq2 is closest to both the moderna and the bioNTech vaccine. Let's make a specific plot of just those three sequences.

In [ ]:
# Now that we know that the vaccine is recognizing a gene in the third largest ORF (index = 2), 
# let's redo the alignent just for the vaccines and for that ORF.
proteinOfInterest = longestORFsequences[2]

seqFile = "sequences.fa"
alignedFile = "aligned_sequences.fa"

# Create a FASTA file with sequences
with open(seqFile, "w") as handle:
    handle.write(f">Moderna\n{ModernaAA}\n")
    handle.write(f">BNTPhi\n{BNTPhiAA}\n")
    handle.write(f">covidProtein\n{proteinOfInterest}\n") 

bashCommand = "muscle -align " + seqFile + " -output " + alignedFile
os.system(bashCommand)

In [34]:
# Make and save an alignment figure
mv = MsaViz(alignedFile, wrap_length=100, show_count=True)
mv.savefig("BioNTech_vs_Moderna_Alignment.png")

In [ ]:
# Load the image
img = mpimg.imread('BioNTech_vs_Moderna_Alignment.png')

# Display the image
fig = plt.figure(figsize=(80, 80), dpi=200)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()

### Let's examine the sequences of the protein we found.

In [ ]:
# Calculate amino acid count and plot as a bar chart
aa_counts = ProtParam.ProteinAnalysis(proteinOfInterest).count_amino_acids()
plt.bar(aa_counts.keys(), aa_counts.values())
plt.xlabel('Amino Acids')
plt.ylabel('Count')
plt.title('Amino Acid Content')
plt.show()

# Calculate amino acid fractions of the protein
amino_acid_composition = ProtParam.ProteinAnalysis(proteinOfInterest).get_amino_acids_percent()
print(f'The amino acid fractions of the protein are:')
for key, value in amino_acid_composition.items():
    print(f"{key}: {value}")


In [ ]:

# Calculate molecular weight of the protein
molecular_weight = ProtParam.ProteinAnalysis(proteinOfInterest).molecular_weight()

print(f"The Molecular weight of this protein is {molecular_weight} Daltons.")

In [ ]:
dir(ProtParam.ProteinAnalysis(proteinOfInterest))

In [ ]:
SecondaryStructFracs = ProtParam.ProteinAnalysis(proteinOfInterest).secondary_structure_fraction()

print(f'Fraction in Helix: {SecondaryStructFracs[0]}')
print(f'Fraction in Turn: {SecondaryStructFracs[1]}')
print(f'Fraction in Sheet: {SecondaryStructFracs[2]}')